In [1]:
import ee
import geemap
import folium
import matplotlib.pyplot as plt
from radiant_mlhub import Dataset

In [3]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWgGNeimSiE38XpSv9gFHOGYhf6huMfJdtvFKlJELerJXiQl19Gdusg

Successfully saved authorization token.


In [4]:
# Area Of Interest

geojson = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              34.73876953125,
              -13.432366575813747
            ],
            [
              36.826171875,
              -13.432366575813747
            ],
            [
              36.826171875,
              -11.802834233547674
            ],
            [
              34.73876953125,
              -11.802834233547674
            ],
            [
              34.73876953125,
              -13.432366575813747
            ]
          ]
        ]
      }
    }
  ]
}

coords = geojson['features'][0]['geometry']['coordinates']
AOI = ee.Geometry.Polygon(coords)

In [5]:
# Add data to the map
Map = geemap.Map()

#point = ee.Geometry.Point([-87.7719, 41.8799])

image = ee.ImageCollection("LANDSAT/LC08/C01/T1_RT_TOA") \
    .filterBounds(AOI) \
    .filterDate('2020-09-01', '2021-03-30') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4']
}

Map.centerObject(AOI, 7)
Map.addLayer(image, vis_params, "Landsat-8")

Map

Map(center=[-12.6187616422214, 35.78247070312495], controls=(WidgetControl(options=['position', 'transparent_b…

In [7]:
training = image.sample(**{
    'region': AOI,
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(training, {}, 'training', False)

# Instantiate the clusterer and train it.
n_clusters = 3
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

In [8]:
# Classify the image

# Cluster the input using the trained clusterer.
result = image.cluster(clusterer)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'clusters')
Map

Map(bottom=17842.0, center=[-12.6187616422214, 35.78247070312495], controls=(WidgetControl(options=['position'…

In [9]:
# Label the clusters

legend_keys = ['One', 'Two', 'Three']
legend_colors = ['red', 'green', 'yellow']

# Reclassify the map
result = result.remap([0, 1, 2], [1, 2, 3])

Map.addLayer(result, {'min': 1, 'max': 3, 'palette': legend_colors}, 'Labelled clusters')
Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright')

The legend colors must be a list of tuples.


In [10]:
# Visualize

print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

In [11]:
Map

Map(bottom=17842.0, center=[-12.6187616422214, 35.78247070312495], controls=(WidgetControl(options=['position'…